In [2]:
import duckdb
print(duckdb.__version__)

1.2.1


## Ingest data

In [3]:
archive_folder = '/home/vikas/Desktop/HuggingFace/github-repo-enumeration/*.csv'

In [4]:
new_schema = {
                'repo_name': 'VARCHAR',
                'unique_contributors': 'INT',
                'commit_comment_event_count': 'INT',
                'create_event_count': 'INT',
                'delete_event_count': 'INT',
                'fork_event_count': 'INT',
                'gollum_event_count': 'INT',
                'issue_comment_event_count': 'INT',
                'issues_event_count': 'INT',
                'member_event_count': 'INT',
                'public_event_count': 'INT',
                'pull_request_event_count': 'INT',
                'pull_request_review_event_count': 'INT',
                'pull_request_review_comment_event_count': 'INT',
                'pull_request_review_thread_event_count': 'INT',
                'push_event_count': 'INT',
                'release_event_count': 'INT',
                'sponsorship_event_count': 'INT',
                'watch_event_count': 'INT'
            }

In [5]:
def ingest_csv(*cols):

    # Convert column names to a comma-separated string
    col_str = ", ".join(cols) if cols else "*"

    csv_input = duckdb.sql(f"""
                           SELECT {col_str} 
                           FROM read_csv('{archive_folder}', 
                                          header = True, 
                                          delim = ',', 
                                          columns = {new_schema}
                                         )
                           """)

    return csv_input

## Run query

### Count

In [6]:
def get_num_rows():

    one_col = ingest_csv("unique_contributors")

    num_rows = duckdb.sql("""
                          SELECT COUNT(*) FROM one_col
                          """)

    print(num_rows)

    return None    

In [7]:
%%time

get_num_rows()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    375897395 │
└──────────────┘

CPU times: user 2min 38s, sys: 12.1 s, total: 2min 50s
Wall time: 12.4 s


### Filter

In [8]:
def filter_user(user_name):

    all_cols = ingest_csv()

    found_user = duckdb.sql(f"""
                            SELECT * FROM all_cols
                            WHERE repo_name LIKE '%{user_name}%'
                            """)

    return found_user    

In [9]:
# %%time

# print(filter_user("vnegi10"))

### Number of repos per user

In [12]:
def get_repo_count_per_user():

    all_cols = ingest_csv("repo_name")

    group_user = duckdb.sql("""
                            WITH repo_data AS (
                            SELECT DISTINCT repo_name
                            FROM all_cols
                            ),
                            split_repo AS (
                                SELECT 
                                    repo_name, 
                                    SPLIT_PART(repo_name, '/', 1) AS username,
                                    SPLIT_PART(repo_name, '/', 2) AS repository
                                FROM repo_data
                            )
                            SELECT 
                                username,
                                COUNT(repository) AS count_repository
                            FROM split_repo
                            GROUP BY username
                            ORDER BY count_repository DESC
                            LIMIT 50;
                            """)

    return group_user   

In [1]:
# def get_repo_count_per_user_opt():
    
#     all_cols = duckdb.sql(f"""
#         SELECT DISTINCT repo_name
#         FROM read_csv_auto('{archive_folder}', header = True)
#     """)

#     group_user = duckdb.sql("""
#         SELECT 
#             SPLIT_PART(repo_name, '/', 1) AS username,
#             COUNT(SPLIT_PART(repo_name, '/', 2)) AS count_repository
#         FROM all_cols
#         GROUP BY username
#         ORDER BY count_repository DESC
#         LIMIT 50;
#     """)

#     return group_user

In [2]:
#print(get_repo_count_per_user())